240104: Have installed things, environment is LiLab. Can see SG386, NIFPGA, and PulseStreamer. Need to now integrate them into one system to see some ODMR Now.

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import tqdm
import pulsestreamer
import nifpga
import pyvisa
import serial
import io

In [ ]:
#### FPGA Bitfile to use for the ODMR
bitfile_loc="C:/NDController-ver2/FPGA Bitfiles/everythingdaq_FPGATarget2_FPGAESRver5_RELKYtnkXk4.lvbitx"

#### Give the different pulse parameters
aomvolt=0.6 # !!! In VOLTS, ONLY 0-1V
pulsenum=100000 
initt=4000 # Durations in ns ONLY
delayt=950 # Durations in ns ONLY
readt=200 # Durations in ns ONLY
waitt=600 # Durations in ns ONLY
mw_freq = 3.12e9 # Hz
mw_power=0 # dBm

#### Connect to Pulsestreamer and setup the pulses
pulsestreamer_ip='169.254.8.2'
ps=pulsestreamer.PulseStreamer(pulsestreamer_ip)
ps.reset()
aompatt=[(96,aomvolt)] # AOM analog modulation voltage, use to control 532nm laser power
ch0patt=[(initt,1),(waitt,0),(delayt+readt,1),(initt,1),(waitt,0),(delayt+readt,1)] # Keep the laser always ON for CW ODMR
# ch1patt=[(waitt,1), (countt,1), (addlt,1),(waitt,0), (countt,0), (addlt,0),(separationt,0)]  # Microwave
ch2patt=[(initt+delayt+waitt,0), (readt,1), (initt+delayt+waitt,0), (readt,1) ]  # Counting
# seq=ps.createSequence()
# seq.setAnalog(0,aompatt)
# seq.setDigital(0,ch0patt)
# # seq.setDigital(1,ch1patt)
# seq.setDigital(2,ch2patt)
# seq.plot()


#### Open and configure the SG386
rm=pyvisa.ResourceManager()
# # device_list=rm.list_resources()
# # print(device_list)
sg386=rm.open_resource('GPIB0::27::INSTR')
# print(sg386)
print(sg386.query('*IDN?'))
# # print(sg386.query('TIMB?'))
sg386.write('FREQ '+str(mw_freq))
print(sg386.query('FREQ?'))
sg386.write('AMPR '+str(mw_power) )
print(sg386.query('AMPR?'))
# sg386.write('ENBR 1')
# print(sg386.query('ENBR?'))
# sg386.close()

## Open and configure the Piezo Controller
# piezo=serial.Serial('COM4',115200,timeout=1)
# # piezo.write("id?")
# # print(piezo.read(10))
# piezo.close()

# for device in device_list:
#     try:
#         instrument=rm.open_resource(device)
#         print(instrument)
#         print(instrument.query('*IDN?'))
#         instrument.close()
#     except:
#         continue

### Connect to FPGA and read the FIFO Buffer
def getContrast(tau,rfdel,pulsenum):
    with nifpga.Session(bitfile=bitfile_loc, resource="RIO0") as session:
        # sg386.write('FREQ '+str(freq))
        # print(sg386.query('FREQ?'))
        session.reset()
        session.run()
        target2host=session.fifos['FIFO_target2host']
        target2host.configure(pulsenum)
        target2host.start()        
        
        ch1patt=[(delayt+initt+waitt+readt,0),(delayt+initt+waitt-tau-rfdel,0), (tau,1), (readt+rfdel,0)]  # Microwave
        seq=ps.createSequence()        
        seq.setDigital(1,ch1patt)
        seq.setAnalog(0,aompatt)
        seq.setDigital(0,ch0patt)
        seq.setDigital(1,ch1patt)
        seq.setDigital(2,ch2patt)        
        # seq.plot()

        ps.stream(seq,n_runs=int(pulsenum/2))
        # print(target2host.name)
        read_value = target2host.read(pulsenum,1000)
        target2host.stop()
        ps.reset()


        counts=read_value[0]
        contrast=sum(counts[0::2])/sum(counts[1::2])

    return contrast,seq



In [ ]:
# print(len(read_value[0]))
# counts=read_value[0]
# # sume=0
# contrast=sum(counts[0::2])/sum(counts[1::2])
# print(getContrast(70))

taus=np.linspace(6,100,40).astype(int)
print(taus)
avg_contrast=np.zeros(len(taus))
num_avgs=50
for j in tqdm.tqdm(range(num_avgs)):
    contrast=np.zeros(len(taus))
    for i in range(len(taus)):
        contrast[i],seq=getContrast(taus[i],rfdel=20,pulsenum=50000 )
        if i==0 and j==0: seq.plot()
    avg_contrast=(j*avg_contrast+contrast)/(j+1) 
    plt.plot(taus,avg_contrast)  
    plt.show() 

# plt.figure(2)
# plt.plot(taus, avg_contrast, linewidth=2)
# plt.xlabel('Tau (ns)',fontsize=16)
# plt.ylabel('Contrast',fontsize=16)
# plt.xticks(fontsize=14)
# plt.yticks(fontsize=14)


In [ ]:
plt.figure(2)
plt.plot(taus, avg_contrast, linewidth=2)
plt.xlabel('Tau (ns)',fontsize=16)
plt.ylabel('Contrast',fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)


In [ ]:
#### Open and configure the Piezo Controller
# piezo=serial.Serial('COM4',115200,timeout=2)
# # piezo.open()
# piezo.write(b"xvoltage=5")
# print(piezo)

# piezo.close()
# seq.plot()


In [ ]:
# piezo.close()

In [ ]:
# a=[1:1:10]